In [ ]:
# import
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from nltk import WordNetLemmatizer
import gensim
from collections import Counter
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from keras import Sequential
from keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding, SpatialDropout1D
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from datetime import datetime
import re
import unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
from google.colab import drive

# Web Crawler



Science News Explores


In [ ]:
links = []; titles = []; authors = []; dates = []; contents = []

url = 'https://www.snexplores.org/topic/environment'
page_url = '/page/'
pages = 28

for page in np.arange(1, pages+1):
      request = requests.get(url + page_url + str(page))
      soup = BeautifulSoup(request.text, "html.parser")
      h3href = soup.find_all("h3", {"class":"post-item-river__title___OLWRU"})
      for i in h3href:
            ahref = i.find('a').get('href')
            request_link = requests.get(ahref)
            link = request_link.url
            links.append(link)
            print(link)

            article = requests.get(link)
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html.parser")

            # title
            title = soup_article.find('title').get_text()
            titles.append(title)

            # author
            author = soup_article.find('span', {'class':'byline author vcard'}).get_text()
            authors.append(author)

            # date
            date = soup_article.find('time', {'class':'date entry-date byline__published___LoyMY'}).attrs['datetime'][:10]
            dates.append(date)

            # content
            body = soup_article.find('div', {"class":"rich-text rich-text--with-sidebar single__rich-text___DT62t"})
            content = body.find_all('p', { "class" : "" })

            advert = body.find('div', class_='newsletter-signup__message___pemaq')
            if advert is not None:
                  body.find('div', class_='newsletter-signup__message___pemaq').decompose()

            paragraphs = []
            for p in content:
                  paragraph = p.get_text() # string
                  paragraphs.append(paragraph)
                  final = " ".join(paragraphs)
            contents.append(final)

print("DONE!")

In [ ]:
dataset1 = pd.DataFrame({'Title':titles, 'Author':authors, 'Date':dates, 'Link':links, 'Content':contents}); print(f'length: {len(dataset1)}') # 328

Science News Independent


In [ ]:
links = []; titles = []; authors = []; dates = []; contents = []

url = 'https://www.sciencenews.org/topic/environment'
page_url = '/page/'
pages = 32

for page in np.arange(1, pages+1):
      request = requests.get(url + page_url + str(page))
      soup = BeautifulSoup(request.text, "html.parser")
      h3href = soup.find_all("h3", {"class":"post-item-river__title___vyz1w"})
      for i in h3href:
            ahref = i.find('a').get('href')
            request_link = requests.get(ahref)
            link = request_link.url
            links.append(link)
            print(link)

            article = requests.get(link)
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html.parser")

            # title
            keep = soup_article.find('div', {'class':'header-default__terms___-BVta'})
            if keep is not None:
                  title = soup_article.find('h1', {'class':'header-default__title___ychM4'}).get_text()
                  titles.append(title)
            else:
                  title = soup_article.find('h1', {'class':'header-immersive__title___asgYs'}).get_text()
                  titles.append(title)

            # author
            author = soup_article.find('span', {'class':'byline author vcard'}).get_text()
            authors.append(author)

            # date
            date = soup_article.find('time', {'class':'byline__published___LoyMY'}).attrs['datetime'][:10]
            dates.append(date)

            # content
            body = soup_article.find('div', {"class":"rich-text rich-text--with-sidebar single__rich-text___RmCDp"})
            content = body.find_all('p', { "class" : "" })

            paragraphs = []
            for p in content:
                  paragraph = p.get_text()
                  paragraphs.append(paragraph)
                  final = " ".join(paragraphs)
            contents.append(final)

print("DONE!")

In [ ]:
dataset2 = pd.DataFrame({'Title':titles, 'Author':authors, 'Date':dates, 'Link':links, 'Content':contents}); print(f'length: {len(dataset2)}') # 376

Frontiers Science News


In [ ]:
links = []; titles = []; authors = []; dates = []; contents = []

url = 'https://blog.frontiersin.org/category/environment/'
page_url = '/page/'
pages = 3

for page in np.arange(1, pages+1):
      url = 'https://blog.frontiersin.org/category/environment/'
      request = requests.get(url + page_url + str(page))
      soup = BeautifulSoup(request.text, "html.parser")
      h3href = soup.find_all("h3", {"class":"loop-title"})
      for i in h3href:
            ahref = i.find('a').get('href')
            request_link = requests.get(ahref)
            link = request_link.url
            links.append(link)
            print(link)

            article = requests.get(link)
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html.parser")

            # title
            title = i.get_text()
            titles.append(title)

            # author
            author = soup_article.find('div', {'class':'entry clearfix'}).find('p').get_text().split(',')[0][3:]
            authors.append(author)

            # date
            date = soup_article.find('span', {'class':'updated'}).get_text()
            date = datetime.strptime(date, '%B %d, %Y').date()
            dates.append(date)

            # content
            body = soup_article.find('div', {"class":"entry clearfix"})
            if soup_article.find('hr') in body:
                  body.find('hr').find_next('a').decompose()
                  body.find('hr').find_next().find_next('a').decompose()
            body.find_next('p').decompose()
            content = body.find_all('p', { "class" : "" })
            content[-1].decompose()

            paragraphs = []
            for p in content:
                  paragraph = p.get_text()
                  paragraphs.append(paragraph)
                  final = " ".join(paragraphs)
            contents.append(final)

print("DONE!")

In [ ]:
dataset3 = pd.DataFrame({'Title':titles, 'Author':authors, 'Date':dates, 'Link':links, 'Content':contents}); print(f'length: {len(dataset3)}') # 108

The New Daily


In [ ]:
links = []; titles = []; authors = []; dates = []; contents = []

url = 'https://thenewdaily.com.au/life/science/environment'
page_url = '/page/'
pages = 50

for page in np.arange(0, pages+1):
      url = 'https://thenewdaily.com.au/life/science/environment'
      request = requests.get(url + page_url + str(page))
      soup = BeautifulSoup(request.text, "html.parser")
      h3href = soup.find_all("a", {"class":"tnd-thumb__link"})
      for i in h3href:
            ahref = i.attrs['href']
            request_link = requests.get(ahref)
            link = request_link.url
            links.append(link)
            print(link)

            article = requests.get(link)
            article_content = article.content
            soup_article = BeautifulSoup(article_content, "html.parser")

            # title
            title = i.find("h1", {'class':'tnd-thumb__h1'}).get_text()
            titles.append(title)

            # author
            author = soup_article.find('span', {'itemprop':'name'}).get_text()
            authors.append(author)

            # date
            date = soup_article.find("time", {'class':'timeago tnd-time'})
            if date is None:
                  date = soup_article.find("time", {'class':'tnd-time'})
                  if date is None:
                        date = soup_article.find("time", {'class':'article-hero__time'})
            date = date.attrs['datetime'][:10]
            dates.append(date)

            # content
            body = soup_article.find('div', {"class":"tnd-content-style tnd-content-style--article"})
            if body.find("p").find("strong") in body:
                  print('yes')
            content = body.find_all('p')
            content[-1].decompose()

            paragraphs = []
            for p in content:
                  paragraph = p.get_text()
                  paragraphs.append(paragraph)
                  final = " ".join(paragraphs)
            contents.append(final)

print("DONE!")

In [ ]:
dataset4 = pd.DataFrame({'Title':titles, 'Author':authors, 'Date':dates, 'Link':links, 'Content':contents}); print(f'length: {len(dataset4)}') #510

In [ ]:
dataset = pd.concat([dataset1, dataset2, dataset3, dataset4], ignore_index=True); print(f'length: {len(dataset)}') #1322

In [ ]:
dataset.to_excel("MA3831_TP1_2023_A3_Dataset_output.xlsx")

# Data Wrangling and Word Embedding


In [ ]:
#@title
drive.mount('/content/gdrive')
dataset = pd.read_excel(r'/content/gdrive/My Drive/Colab Notebooks/MA3831_TP1_2023_Dataset_output.xlsx')
dataset = dataset.iloc[:, 1:] # drop first column from the dataset

print(f"length before: {len(dataset)}")
dataset = dataset.dropna()
print(f"length after: {len(dataset)}")

In [ ]:
dataset.drop_duplicates(subset=['Title','Author'],keep='first',inplace=True)

dataset['Date'] = dataset['Date'].apply(str).str[:7].str.replace('-', '', regex=True).astype(int)
dataset['content_words'] = dataset['Content'].str.split().str.len()
dataset['title_words'] = dataset['Title'].str.split().str.len()

dataset.drop(dataset[dataset['content_words'] < 200].index, inplace=True)
dataset.drop(dataset[dataset['title_words'] < 6].index, inplace=True)


In [ ]:
dataset['Sentiment'] = dataset['Sentiment'].astype(str)
dataset['Sentiment'] = dataset['Sentiment'].str.replace('-1', '0').astype(int)
print("Number of rows per sentiment rating:")
print(dataset['Sentiment'].value_counts())
ax = dataset.groupby('Sentiment').count().plot(kind='bar', legend=False, color='black')
ax = ax.set_xticklabels(['Negative','Positive'], rotation=0)

# Function to retrieve top few number of each category
top_n = 500

def get_top_data(top_n):
    top_data_df_positive = dataset[dataset['Sentiment'] == 1].head(top_n)
    top_data_df_negative = dataset[dataset['Sentiment'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative])
    return top_data_df_small

# Function call to get the top 10000 from each sentiment
dataset = get_top_data(top_n=top_n)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(dataset['Sentiment'].value_counts())

In [ ]:
# Normalising Text
def normalise_text(string_text):
  if isinstance(string_text, str):
    #replace lower format
    string_text = string_text.lower()
    #remove special characters
    pattern = r'''(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?|(?:[A-Z]\.)|\$?\d+(?:\.\d+)?%?|\.\.\.| [][.,;"'?():_'-]'''
    string_text = re.sub(pattern, ' ', string_text)
    #remove accented characters
    string_text = unicodedata.normalize('NFKD', string_text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #remove stopwords
    stopword_list = nltk.corpus.stopwords.words('english')
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(string_text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    string_text = ' '.join(filtered_tokens)
    return string_text


def lemmatize_text(string_text):
  wnl = WordNetLemmatizer()
  string_text = nltk.word_tokenize(string_text)
  string_text = ' '.join(wnl.lemmatize(token) for token in string_text)
  return string_text

In [ ]:
features = ['Title', 'Author', 'Content']

for feature in features:
  dataset[feature] = dataset[feature].apply(lemmatize_text)
  dataset[feature] = dataset[feature].apply(normalise_text)
  dataset[feature] = dataset[feature].apply(gensim.utils.simple_preprocess)

In [ ]:
def extract_topics():
    topic_list = []
    for title, content in zip(dataset.Title, dataset.Content):
      results = {}
      for string in title:
        results[string] = content.count(string)
      string = max(results, key=results.get)
      topic_list.append(string)
    return topic_list

topics = extract_topics()
dataset['Topic'] = topics

# for text in dataset.Topic:
common_topics = Counter(" ".join(topics).split()).most_common(10)
print(common_topics)

In [ ]:
def wordcloud_plot(topics):
  #plot the word cloud object
  text = ' '.join(topics)

  wordcloud = WordCloud(stopwords = STOPWORDS, collocations=True, min_word_length=5).generate(text)
  plt.imshow(wordcloud, interpolation='bilInear')
  plt.axis('off')
  plt.show()

  # create a dictionary of word frequencies
  text_dictionary = wordcloud.process_text(text)
  # sort the dictionary
  word_freq={k: v for k, v in sorted(text_dictionary.items(),reverse=True, key=lambda item: item[1])}
  #use words_ to print relative word frequencies
  rel_freq = wordcloud.words_
  #print results
  print(list(word_freq.items())[:15])
  print(list(rel_freq.items())[:10])

In [ ]:
#negative word cloud
negative_topics = dataset[dataset.Sentiment == 0]
negative_topics = negative_topics['Topic']

wordcloud_plot(negative_topics)

In [ ]:
#positive word cloud
positive_topics = dataset[dataset.Sentiment == 1]
positive_topics = positive_topics['Topic']

wordcloud_plot(positive_topics)

In [ ]:
#training and testing split data
X_data, y_data = np.array(dataset['Content']), np.array(dataset['Sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.05, random_state = 0)

In [ ]:
Word2vec_train_data = list(map(lambda x: x, X_train))

Embedding_dimensions = 500 
window = 5

word2vec_model  = Word2Vec(Word2vec_train_data,
                     min_count=1,
                     vector_size=Embedding_dimensions,
                     window=window,
                     sg=0)

print("Vocabulary Length:", len(word2vec_model.wv.key_to_index))

In [ ]:
print(word2vec_model.wv.most_similar(positive=['climate'], topn=3))

In [ ]:
keys = ['climate', 'plastic', 'water', 'air', 'oil']

embedding_clusters = []
word_clusters = []
for word in keys:
    embeddings = []
    words = []
    for similar_word, words_ in word2vec_model.wv.most_similar(word, topn=30):
        words.append(similar_word)
        embeddings.append(word2vec_model.wv[similar_word])
    embedding_clusters.append(embeddings)
    word_clusters.append(words)

tsne_model_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32)
embedding_clusters = np.array(embedding_clusters)
n, m, k = embedding_clusters.shape
embeddings_en_2d = np.array(tsne_model_en_2d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 2)


def tsne_plot_similar_words(labels, embedding_clusters, word_clusters, a=0.7):
    plt.figure(figsize=(19, 9))
    # colors = cm.rainbow(np.linspace(0, 1, len(labels)))
    colors = ['green', 'orange', 'blue', 'red', 'black']
    for label, embeddings, words, color in zip(labels, embedding_clusters, word_clusters, colors):
        x = embeddings[:,0]
        y = embeddings[:,1]
        plt.scatter(x, y, c=color, alpha=a, label=label)
        for i, word in enumerate(words):
            plt.annotate(word, alpha=0.5, xy=(x[i], y[i]), xytext=(5, 5),
                         textcoords='offset points', ha='right', va='bottom', size=9)
    plt.legend(loc=4)
    plt.grid(True)
    plt.show()


tsne_plot_similar_words(keys, embeddings_en_2d, word_clusters)

for word in keys:
    positive_words = word2vec_model.wv.most_similar(positive=[word], topn=3)
    negative_words = word2vec_model.wv.most_similar(negative=[word], topn=3)

    print(f"{word}: {positive_words} \n")

#Deep Learning

In [ ]:
# Defining the model input length.
input_length = 1000
vocab_length = 30000

tokenizer = Tokenizer(filters="", lower=False, oov_token="<oov>")
tokenizer.fit_on_texts(X_data)
tokenizer.num_words = vocab_length
print("Tokenizer vocab length:", vocab_length)

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test  = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=input_length)

In [ ]:
embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))

for word, token in tokenizer.word_index.items():
    if word2vec_model.wv.__contains__(word):
      embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

In [ ]:
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length,
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix],
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(200, dropout=0.5, return_sequences=True)),
        Bidirectional(LSTM(150, dropout=0.2, return_sequences=True)),
        GlobalMaxPool1D(),
        Dense(24, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model

In [ ]:
training_model = getModel()
training_model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
             EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5)]

training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])         

In [ ]:
history = training_model.fit(
    X_train, y_train,         
    batch_size=300,
    # batch_size=256, # 0.69 (winner!)
    epochs=50,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
# acc,  val_acc  = history.history['accuracy'], history.history['val_accuracy']
# loss, val_loss = history.history['loss'], history.history['val_loss']
# epochs = range(len(acc))


# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.legend()

# plt.figure()

# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.legend()

# plt.show()

# plt.plot(history.history['loss'])
# plt.plot(history.history['accuracy'])
# plt.title('model loss vs accuracy')
# plt.xlabel('epoch')
# plt.legend(['loss', 'accuracy'], loc='upper right')
# plt.show() 

In [ ]:
def ConfusionMatrix(y_pred, y_test):
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(y_test, y_pred)

    categories  = ['Negative','Positive']
    group_names = ['True Neg','False Pos', 'False Neg','True Pos']
    group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted values", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual values"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
# Predicting on the Test dataset.
y_pred = training_model.predict(X_test)

# Converting prediction to reflect the sentiment predicted.
y_pred = np.where(y_pred>=0.5, 1, 0)

# Printing out the Evaluation metrics. 
ConfusionMatrix(y_pred, y_test)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred)
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]

In [ ]:
# Print the evaluation metrics for the dataset.
print(classification_report(y_test, y_pred))

In [ ]:
keys = ['climate', 'plastic', 'water', 'air', 'oil']
df = dataset[(dataset.Year >= 2022) & (dataset['Topic'].isin(keys))]

In [ ]:
sentiment_list = []
for data in df['Content']:
  twt = data
  #vectorizing the tweet by the pre-fitted tokenizer instance
  twt = tokenizer.texts_to_sequences(twt)
  #padding the tweet to have exactly the same shape as `embedding_2` input
  twt = pad_sequences(twt, maxlen=500, dtype='int32', value=0)
  sentiment = training_model.predict(twt,batch_size=1,verbose = 2)
  print(np.average(sentiment))
  if (np.average(sentiment) <= 0.5):
    predict_sentiment = 0
  else:
    predict_sentiment = 1
  sentiment_list.append(predict_sentiment)

df['predicted_sentiment'] = sentiment_list

In [ ]:
# Cross tabulation between Topic and predicted_sentiment
CrosstabResult=pd.crosstab(index=df['Topic'],columns=df['predicted_sentiment'])
print(CrosstabResult)
 
# Grouped bar chart between Topic and predicted_sentiment
CrosstabResult.plot.bar(figsize=(7,4), rot=0, color={0:'grey',1:'black'})
plt.legend(["Negative","Positive"])